# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
# Download link 1
# !gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

# !unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Sun Apr 30 22:27:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.68                 Driver Version: 531.68       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P8               19W /  N/A|    974MiB / 16384MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
# !pip install transformers==4.5.0

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
# from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from torch.optim import AdamW
from transformers import BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, get_cosine_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(0)

In [4]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True # False

if fp16_training:
    # !pip install accelerate==0.2.0
    from accelerate import Accelerator
    # accelerator = Accelerator(fp16=True)
    accelerator = Accelerator(mixed_precision="fp16")
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [5]:
model_name = "luhua/chinese_pretrain_mrc_macbert_large" # "hfl/chinese-roberta-wwm-ext" # "bert-base-chinese"

model = BertForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [6]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [7]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [8]:
MAX_PARAGRAPH_LEN = 469
DOC_STRIDE = MAX_PARAGRAPH_LEN // 5

In [9]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = MAX_PARAGRAPH_LEN # 150
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = DOC_STRIDE # 75 # 150

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            start_min = max(0, answer_end_token - self.max_paragraph_len + 1)
            start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)
            start_max = max(start_min, start_max)
            paragraph_start = random.randint(start_min, start_max + 1)
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8 # 16 # 32

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [10]:
def evaluate(data, output, raw_paragraph=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        if start_index > end_index:
            continue
        
        sep_pos = np.where(data[0][0][k] == 102)[0][0] + 1
        
        if start_index < sep_pos:
            continue
        
        # tmp_ans = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
        # tmp_ans = tmp_ans.replace(" ", "")
        
        # if "[CLS]" in tmp_ans or "[SEP]" in tmp_ans:
        #     continue
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        # prob = start_prob * end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            answer = answer.replace(" ", "")
            # answer = tmp_ans
        
            if "[UNK]" in answer and raw_paragraph is not None:
                # sep_pos = np.where(data[0][0][k] == 102)[0][0] + 1
                raw_paragraph_tokenized = tokenizer(raw_paragraph, add_special_tokens=False)
                raw_start_index = raw_paragraph_tokenized.token_to_chars(k * DOC_STRIDE + start_index - sep_pos)[0]
                raw_end_index = raw_paragraph_tokenized.token_to_chars(k * DOC_STRIDE + end_index - sep_pos)[1]
                answer = raw_paragraph[raw_start_index:raw_end_index]
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    # return answer.replace(' ','')
    return answer

## Training

In [11]:
num_epoch = 10
validation = True
logging_step = 100
learning_rate = 1e-5
grad_acc_step = 4
early_stop_patience = 1
model_save_dir = "saved_model" 

In [12]:
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=3e-2)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_epoch * len(train_loader) // 3, num_epoch * len(train_loader))
# scheduler = get_linear_schedule_with_warmup(optimizer, num_epoch * len(train_loader) // 2, num_epoch * len(train_loader))
# scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader) // 2,
#                                                                num_training_steps=num_epoch * len(train_loader), num_cycles=num_epoch+1)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")
best_dev_acc = 0.0
early_stop_count = 0

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        if step % grad_acc_step == 0:
            optimizer.step()
            optimizer.zero_grad()
        scheduler.step()
        step += 1

        ##### TODO: Apply linear learning rate decay #####
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | learning rate = {optimizer.param_groups[0]['lr']:.3e}, loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, dev_paragraphs[dev_questions[i]["paragraph_id"]]) == dev_questions[i]["answer_text"]
            dev_acc /= len(dev_loader)
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc:.3f}")
        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            print("Saving best model so far ...")
            model.save_pretrained(model_save_dir)
            early_stop_count = 0
        else:
            early_stop_count += 1
            
        if early_stop_count > early_stop_patience:
            print(f"No improvements in the recent {early_stop_patience} epochs. So far the best model with validation acc={best_dev_acc:.3f}.")
            break
            
        model.train()

# # Save a model and its configuration file to the directory 「saved_model」 
# # i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# # Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
# print("Saving Model ...")
# model_save_dir = "saved_model" 
# model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 1 | Step 100 | learning rate = 7.497e-08, loss = 2.357, acc = 0.412
Epoch 1 | Step 200 | learning rate = 1.507e-07, loss = 2.240, acc = 0.455
Epoch 1 | Step 300 | learning rate = 2.264e-07, loss = 2.027, acc = 0.450
Epoch 1 | Step 400 | learning rate = 3.021e-07, loss = 1.773, acc = 0.505
Epoch 1 | Step 500 | learning rate = 3.779e-07, loss = 1.583, acc = 0.536
Epoch 1 | Step 600 | learning rate = 4.536e-07, loss = 1.437, acc = 0.571
Epoch 1 | Step 700 | learning rate = 5.293e-07, loss = 1.150, acc = 0.591
Epoch 1 | Step 800 | learning rate = 6.050e-07, loss = 1.122, acc = 0.615
Epoch 1 | Step 900 | learning rate = 6.808e-07, loss = 1.137, acc = 0.625
Epoch 1 | Step 1000 | learning rate = 7.565e-07, loss = 1.057, acc = 0.636
Epoch 1 | Step 1100 | learning rate = 8.322e-07, loss = 0.950, acc = 0.656
Epoch 1 | Step 1200 | learning rate = 9.079e-07, loss = 0.829, acc = 0.678
Epoch 1 | Step 1300 | learning rate = 9.836e-07, loss = 0.889, acc = 0.673
Epoch 1 | Step 1400 | learning rat

  0%|          | 0/4131 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.805
Saving best model so far ...


...

Validation | Epoch 4 | acc = 0.817


  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 5 | Step 100 | learning rate = 9.737e-06, loss = 0.102, acc = 0.946
Epoch 5 | Step 200 | learning rate = 9.717e-06, loss = 0.094, acc = 0.952
Epoch 5 | Step 300 | learning rate = 9.697e-06, loss = 0.106, acc = 0.939
Epoch 5 | Step 400 | learning rate = 9.677e-06, loss = 0.123, acc = 0.939
Epoch 5 | Step 500 | learning rate = 9.655e-06, loss = 0.100, acc = 0.954
Epoch 5 | Step 600 | learning rate = 9.633e-06, loss = 0.089, acc = 0.957
Epoch 5 | Step 700 | learning rate = 9.610e-06, loss = 0.117, acc = 0.942
Epoch 5 | Step 800 | learning rate = 9.587e-06, loss = 0.112, acc = 0.939
Epoch 5 | Step 900 | learning rate = 9.563e-06, loss = 0.074, acc = 0.964
Epoch 5 | Step 1000 | learning rate = 9.538e-06, loss = 0.094, acc = 0.946
Epoch 5 | Step 1100 | learning rate = 9.513e-06, loss = 0.094, acc = 0.945
Epoch 5 | Step 1200 | learning rate = 9.487e-06, loss = 0.103, acc = 0.959
Epoch 5 | Step 1300 | learning rate = 9.461e-06, loss = 0.095, acc = 0.944
Epoch 5 | Step 1400 | learning rat

  0%|          | 0/4131 [00:00<?, ?it/s]

Validation | Epoch 5 | acc = 0.805
No improvements in the recent 1 epochs. So far the best model with validation acc=0.817.


## Testing

In [13]:
print("Evaluating Test Set ...")

model.from_pretrained(model_save_dir)

result = []

model.eval()
with torch.no_grad():
    i = 0
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        ans = evaluate(data, output, test_paragraphs[test_questions[i]["paragraph_id"]])
        result.append(ans)
        i += 1

result_file = "result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

Completed! Result is in result.csv
